In [1]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
INPUT_FOLDER = '../DaneDICOM/DICOMNativeAnonymized/'
over_patients = os.listdir(INPUT_FOLDER)
# patients.sort()
print(over_patients, len(over_patients))

ModuleNotFoundError: No module named 'pydicom'

In [ ]:
# sample_patient = os.listdir(INPUT_FOLDER + over_patients[0])[0]
# sample_patient = INPUT_FOLDER + over_patients[0] + '/' + sample_patient
# print(sample_patient)
# #sample_slices = pydicom.read_file(sample_patient)

In [2]:
def get_patient_target_folder(patient_parent_folder):
    patient_target_folder = os.listdir(patient_parent_folder)[0]
    patient_target_folder = patient_parent_folder + '/' + patient_target_folder
    return patient_target_folder

In [3]:
def get_list_patients_paths(input_folder):
    list_patients_paths = []
    for element in os.listdir(input_folder):
        patient_parent_folder = os.path.join(INPUT_FOLDER, element)
        patient_target_folder = get_patient_target_folder(patient_parent_folder)
        list_patients_paths.append(patient_target_folder)
        # print(patient_target_folder)
    return list_patients_paths

list_patients_paths = get_list_patients_paths(INPUT_FOLDER)
print(list_patients_paths)


['../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110301.154346.9227917/1.3.12.2.1107.5.1.4.53642.30000011022806271114000017842', '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110321.143718.22145985/1.3.12.2.1107.5.1.4.53642.30000011032114095153100000518', '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110513.80519.17850612/1.3.12.2.1107.5.1.4.53642.30000011051219413425000001288', '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110524.151401.25288183/1.3.12.2.1107.5.1.4.53642.30000011052316233285900008053', '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110524.92951.24933278/1.3.12.2.1107.5.1.4.53642.30000011052316233285900003826', '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110728.95808.26592756/1.3.12.2.1107.5.1.4.53642.30000011072805541090600004108', '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110729.100309.27384465/1.3.12.2.1107.5.1.

In [4]:
# source: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
# Load the scans in given folder path
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [5]:
# source: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [6]:
sample_patient = list_patients_paths[1]
sample_patient = '../DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110321.143718.22145985/1.3.12.2.1107.5.1.4.53642.30000011032114095153100000518'
sample_patient = 'C:/Users/User/Documents/DaneDICOM/DICOMNativeAnonymized/1.2.124.113532.12.10699.12127.20110321.143718.22145985/1.3.12.2.1107.5.1.4.53642.30000011032114095153100000518'
first_patient = load_scan(sample_patient)
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Show some slice in the middle
plt.imshow(first_patient_pixels[0], cmap=plt.cm.gray)
plt.show()

OSError: cannot identify image file <_io.BytesIO object at 0x00000159A4B6C168>

In [31]:
#print(len(list_patients_paths))
for patient_path in list_patients_paths:
    slices = []
    slices = [pydicom.read_file(patient_path + '/' + s) for s in os.listdir(patient_path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        print(slices[0].pixel_array.shape)
    except:
        print("Pixel array not found")
        #plt.imshow(slices[80].pixel_array)
    #plt.show()

(512, 512)
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found
Pixel array not found

FileNotFoundError: [Errno 2] No such file or directory: '../DaneDICOM/DICOMNativeAnonymized/1.2.392.200036.9116.2.2.2.1762658420.1203904329.336701/1.2.392.200036.9116.2.2.2.1762658420.1203904462.52110/1.2.392.200036.9116.2.2.2.1762658420.1203904462.52110_0002_000001_15526461190025.dcm'

In [ ]:
# source: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness] + scan[0].PixelSpacing))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

In [ ]:
# pix_resampled, spacing = resample(first_patient_pixels, first_patient, [1,1,1])
# print("Shape before resampling\t", first_patient_pixels.shape)
# print("Shape after resampling\t", pix_resampled.shape)

In [ ]:
def plot_3d(image, threshold=-300):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    p = p[:,:,::-1]
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [ ]:
plot_3d(first_patient_pixels, 400)